In [1]:
import os,sys,warnings,re,math,gc,time
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["KERAS_BACKEND"] = "tensorflow"
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from sklearn.metrics import roc_auc_score,auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold,StratifiedGroupKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import tensorflow as tf
import keras
tf.get_logger().setLevel("ERROR")
%xmode Minimal

Exception reporting mode: Minimal


In [2]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_thousands_separator(",")
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(6)

polars.config.Config

In [3]:
path_to_train = "Downloads/credit_risk/train"
path_to_test = "Downloads/credit_risk/test"
path_to_features = "Downloads/feature_definitions.csv"

In [4]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [5]:
all_train_files = glob(path_to_train+"/*.parquet")
all_test_files = glob(path_to_test+"/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [6]:
train_base = (
    pl.read_parquet(path_to_train+"/train_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
train_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
0,2019-01-03,1,1,4,0,0
1,2019-01-03,1,1,4,0,0
2,2019-01-04,1,1,5,0,0
…,…,…,…,…,…,…
"2,703,452",2020-10-05,10,41,1,1,0
"2,703,453",2020-10-05,10,41,1,1,0
"2,703,454",2020-10-05,10,41,1,1,0


In [7]:
test_base = (
    pl.read_parquet(path_to_test+"/test_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
)
test_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
"57,543",2020-10-06,10,41,2,1,0
"57,549",2020-10-06,10,41,2,1,0
"57,551",2020-10-06,10,41,2,1,0
…,…,…,…,…,…,…
"57,632",2020-10-06,10,41,2,1,0
"57,633",2020-10-06,10,41,2,1,0
"57,634",2020-10-06,10,41,2,1,0


In [8]:
total_base = (
    pl.concat(
        [
            train_base,
            test_base
        ],
        how="vertical"
    )
)

In [9]:
def reduce_dtypes(df:pl.DataFrame):
    return (
        df
        .with_columns(
            cs.by_name("case_id").cast(pl.UInt32),
            cs.contains("num_group").cast(pl.UInt16),
            cs.ends_with("D").cast(pl.Date),
            cs.ends_with("T","M").cast(pl.String),
            cs.ends_with("P","A").cast(pl.Float32),
            (cs.ends_with("L") & cs.numeric()).cast(pl.Float32),
        )
    )

def grouping(df):
    return (
        df
        .group_by("case_id")
        .agg(
            cs.numeric().max(),
            (~cs.numeric()).drop_nulls().mode().first()
        )
    )

def preprocess(filter_string:str,prefix_string:str):
    train_files_list = train_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    test_files_list = test_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    with pl.StringCache():
        train_df = (
            pl.concat(
                [
                    pl.read_parquet(_).pipe(reduce_dtypes).pipe(grouping) for _ in train_files_list
                ]
            )
        )
        test_df = (
            pl.concat(
                [
                    pl.read_parquet(_).select(train_df.columns).cast(train_df.schema).pipe(grouping) for _ in test_files_list
                ]
            )
        )
    return (
        pl.concat(
            [
                train_df,
                test_df
            ],
            how="vertical_relaxed"
        )
        .pipe(lambda df: df.rename({_:f"{prefix_string}_{_}" for _ in df.columns if not _ == 'case_id'}))
    )


def select_impuatable(df:pl.DataFrame,thresh=0.95):
    cols =  (
        df
        .select(pl.all().is_null().mean())
        .transpose(include_header=True)
        .filter(pl.col("column_0") < thresh)
        ["column"].to_list()
    )
    return df.select(cols)

In [10]:
total_past_shallow = preprocess("applprev_1","past_shallow")
total_past_shallow

In [ ]:
total_past_depth = preprocess("applprev_2","past_depth")
total_past_depth

case_id,past_depth_num_group1,past_depth_num_group2,past_depth_cacccardblochreas_147M,past_depth_conts_type_509L,past_depth_credacc_cards_status_52L
u32,u16,u16,str,str,str
"1,434,046",2,3,"""a55475b1""","""PRIMARY_MOBILE""",null
"1,487,381",5,2,"""a55475b1""","""PRIMARY_MOBILE""",null
"1,352,644",1,2,"""a55475b1""","""PRIMARY_MOBILE""",null
…,…,…,…,…,…
"57,543",0,0,null,"""PRIMARY_MOBILE""",null
"57,551",1,1,null,"""PRIMARY_MOBILE""",null
"57,552",0,2,null,"""PRIMARY_MOBILE""",null


In [ ]:
total_static_base = preprocess("static_0","static_base")
total_static_base

case_id,static_base_actualdpdtolerance_344P,static_base_amtinstpaidbefduel24m_4187115A,static_base_annuity_780A,static_base_annuitynextmonth_57A,static_base_applicationcnt_361L,static_base_applications30d_658L,static_base_applicationscnt_1086L,static_base_applicationscnt_464L,static_base_applicationscnt_629L,static_base_applicationscnt_867L,static_base_avgdbddpdlast24m_3658932P,static_base_avgdbddpdlast3m_4187120P,static_base_avgdbdtollast24m_4525197P,static_base_avgdpdtolclosure24_3658938P,static_base_avginstallast24m_3658937A,static_base_avglnamtstart24m_4525187A,static_base_avgmaxdpdlast9m_3716943P,static_base_avgoutstandbalancel6m_4187114A,static_base_avgpmtlast12m_4525200A,static_base_clientscnt12m_3712952L,static_base_clientscnt3m_3712950L,static_base_clientscnt6m_3712949L,static_base_clientscnt_100L,static_base_clientscnt_1022L,static_base_clientscnt_1071L,static_base_clientscnt_1130L,static_base_clientscnt_136L,static_base_clientscnt_157L,static_base_clientscnt_257L,static_base_clientscnt_304L,static_base_clientscnt_360L,static_base_clientscnt_493L,static_base_clientscnt_533L,static_base_clientscnt_887L,static_base_clientscnt_946L,static_base_cntincpaycont9m_3716944L,…,static_base_credtype_322L,static_base_datefirstoffer_1144D,static_base_datelastinstal40dpd_247D,static_base_datelastunpaid_3546854D,static_base_disbursementtype_67L,static_base_dtlastpmtallstes_4499206D,static_base_equalitydataagreement_891L,static_base_equalityempfrom_62L,static_base_firstclxcampaign_1125D,static_base_firstdatedue_489D,static_base_inittransactioncode_186L,static_base_isbidproduct_1095L,static_base_isbidproductrequest_292L,static_base_isdebitcard_729L,static_base_lastactivateddate_801D,static_base_lastapplicationdate_877D,static_base_lastapprcommoditycat_1041M,static_base_lastapprcommoditytypec_5251766M,static_base_lastapprdate_640D,static_base_lastcancelreason_561M,static_base_lastdelinqdate_224D,static_base_lastrejectcommoditycat_161M,static_base_lastrejectcommodtypec_5251769M,static_base_lastrejectdate_50D,static_base_lastrejectreason_759M,static_base_lastrejectreasonclient_4145040M,static_base_lastrepayingdate_696D,static_base_lastst_736L,static_base_maxdpdinstldate_3546855D,static_base_opencred_647L,static_base_paytype1st_925L,static_base_paytype_783L,static_base_payvacationpostpone_4187118D,static_base_previouscontdistrict_112M,static_base_twobodfilling_608L,static_base_typesuite_864L,static_base_validfrom_1069D
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,str,date,date,date,str,date,bool,bool,date,date,str,bool,bool,bool,date,date,str,str,date,str,date,str,str,date,str,str,date,str,date,bool,str,str,date,str,str,str,date
"1,300,184",0,null,"4,581",0,0,0,0,0,0,1,0,null,null,0,"9,029",null,null,null,null,0,0,0,0,0,0,0,null,0,0,0,0,0,0,0,0,0,…,"""COL""",null,null,2018-05-29,"""SBA""",null,null,null,null,2017-12-29,"""POS""",false,null,null,2018-01-29,2017-11-29,"""P159_130_59""","""a55475b1""",2017-11-29,"""a55475b1""",2018-05-29,"""a55475b1""","""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""K""",2018-05-29,false,"""OTHER""","""OTHER""",null,"""P192_103_107""","""FO""",null,null
"1,492,892",0,"21,747","1,497.800048828125",0,0,0,0,0,0,2,-2,null,null,0,"2,145.400146484375",null,0,null,null,0,0,0,0,0,0,0,null,0,0,0,0,0,0,0,0,1,…,"""COL""",2016-03-01,null,2018-12-05,"""SBA""",2018-12-06,null,null,2016-03-01,2015-05-11,"""POS""",false,null,null,2018-01-11,2017-12-05,"""P159_130_59""","""a55475b1""",2017-12-05,"""a55475b1""",2018-12-05,"""a55475b1""","""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""K""",2018-02-10,false,"""OTHER""","""OTHER""",null,"""P98_137_111""","""FO""",null,null
"1,631,428",0,"12,377",458.3999938964844,"1,579.5999755859375",0,0,0,0,0,4,-5,-5,-5,0,"1,074.5999755859375","9,996",0,"28,420.373046875","1,074.5999755859375",0,0,0,0,1,0,0,null,0,0,0,0,0,0,0,0,10,…,"""COL""",null,null,null,"""SBA""",201

In [ ]:
total_static_external = (
    preprocess("static_cb","static_external")
    .with_columns(
        pl.col("static_external_riskassesment_302T").str.split("%").list.gather([0,1]).apply(lambda x: (int(x[0])+int(x[1].split("-")[1]))/200)
    )
)
total_static_external

case_id,static_external_contractssum_5085716L,static_external_days120_123L,static_external_days180_256L,static_external_days30_165L,static_external_days360_512L,static_external_days90_310L,static_external_firstquarter_103L,static_external_for3years_128L,static_external_for3years_504L,static_external_for3years_584L,static_external_formonth_118L,static_external_formonth_206L,static_external_formonth_535L,static_external_forquarter_1017L,static_external_forquarter_462L,static_external_forquarter_634L,static_external_fortoday_1092L,static_external_forweek_1077L,static_external_forweek_528L,static_external_forweek_601L,static_external_foryear_618L,static_external_foryear_818L,static_external_foryear_850L,static_external_fourthquarter_440L,static_external_numberofqueries_373L,static_external_pmtaverage_3A,static_external_pmtaverage_4527227A,static_external_pmtaverage_4955615A,static_external_pmtcount_4527229L,static_external_pmtcount_4955617L,static_external_pmtcount_693L,static_external_pmtscount_423L,static_external_pmtssum_45A,static_external_secondquarter_766L,static_external_thirdquarter_1082L,static_external_assignmentdate_238D,static_external_assignmentdate_4527235D,static_external_assignmentdate_4955616D,static_external_birthdate_574D,static_external_dateofbirth_337D,static_external_dateofbirth_342D,static_external_description_5085714M,static_external_education_1103M,static_external_education_88M,static_external_maritalst_385M,static_external_maritalst_893M,static_external_requesttype_4525192L,static_external_responsedate_1012D,static_external_responsedate_4527233D,static_external_responsedate_4917613D,static_external_riskassesment_302T,static_external_riskassesment_940T
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,date,date,date,date,date,date,str,str,str,str,str,str,date,date,date,f64,str
"1,783,085",null,7,10,4,16,7,17,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6,16,null,null,null,null,null,null,null,null,10,10,null,null,null,null,1978-03-01,null,"""a55475b1""","""6b2ae0fa""","""a55475b1""","""3439d993""","""a55475b1""","""DEDUCTION_6""",null,2020-02-26,null,null,null
"1,282,621",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"18,319.732421875",null,null,null,null,6,null,null,null,null,2008-04-09,null,null,1950-04-01,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,2019-02-28,null,null,null,null
"112,470",null,3,3,0,6,3,5,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5,6,null,null,null,null,null,null,7,"10,208.7998046875",4,1,null,null,null,1990-10-01,1990-10-01,null,"""a55475b1""","""6b2ae0fa""","""a55475b1""","""3439d993""","""a55475b1""",null,2019-03-07,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,552",null,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0,0,null,null,"16,327",null,14,null,null,null,0,0,null,null,2012-03-02,null,1948-10-01,null,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,2020-10-19,null,null
"57,634",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"6,917",null,14,null,null,null,null,null,null,null,2017-12-15,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,2020-10-20,null,null
"57,631","750,875.4375",6,7,1,12,1,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,12,null,null,"16,863",null,12,null,null,null,6,0,null,null,2013-04-12,null,1955-03-01,null,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,2020-10-20,null,null


In [ ]:
total_person_shallow = preprocess("person_1","person_shallow")
total_person_shallow

case_id,person_shallow_childnum_185L,person_shallow_mainoccupationinc_384A,person_shallow_num_group1,person_shallow_personindex_1023L,person_shallow_persontype_1072L,person_shallow_persontype_792L,person_shallow_birth_259D,person_shallow_birthdate_87D,person_shallow_contaddr_district_15M,person_shallow_contaddr_matchlist_1032L,person_shallow_contaddr_smempladdr_334L,person_shallow_contaddr_zipcode_807M,person_shallow_education_927M,person_shallow_empl_employedfrom_271D,person_shallow_empl_employedtotal_800L,person_shallow_empl_industry_691L,person_shallow_empladdr_district_926M,person_shallow_empladdr_zipcode_114M,person_shallow_familystate_447L,person_shallow_gender_992L,person_shallow_housetype_905L,person_shallow_housingtype_772L,person_shallow_incometype_1044T,person_shallow_isreference_387L,person_shallow_language1_981M,person_shallow_maritalst_703L,person_shallow_registaddr_district_1083M,person_shallow_registaddr_zipcode_184M,person_shallow_relationshiptoclient_415T,person_shallow_relationshiptoclient_642T,person_shallow_remitter_829L,person_shallow_role_1084L,person_shallow_role_993L,person_shallow_safeguarantyflag_411L,person_shallow_sex_738L,person_shallow_type_25L
u32,f32,f32,u16,f32,f32,f32,date,date,str,bool,bool,str,str,date,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,bool,str,str,bool,str,str
"753,462",null,"30,000",2,1,5,5,1994-09-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-04-15,"""LESS_ONE""","""EDUCATION""","""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""PRIVATE_SECTOR_EMPLOYEE""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""PARENT""","""PARENT""",false,"""CL""",null,true,"""M""","""PHONE"""
"807,455",null,"80,000",2,1,4,4,1996-01-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2018-08-15,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""PRIVATE_SECTOR_EMPLOYEE""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SPOUSE""","""SPOUSE""",false,"""PE""",null,true,"""F""","""PRIMARY_MOBILE"""
"3,933",null,"40,200",2,2,5,5,1957-04-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""RETIRED_PENSIONER""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""CHILD""","""CHILD""",false,"""PE""",null,true,"""F""","""PHONE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,551",null,"24,000",2,1,5,5,1990-08-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-01-15,"""MORE_FIVE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""FRIEND""","""FRIEND""",false,"""EM""",null,false,"""M""","""PHONE"""
"57,549",null,"15,000",3,1,5,5,1992-03-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-05-04,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SIBLING""","""SIBLING""",false,"""EM""",null,false,"""M""","""PRIMARY_MOBILE"""
"57,543",null,"36,000",2,1,5,5,1996-08-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2018-02-15,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""OTHER""","""OTHER""",false,"""CL""",null,false,"""M""","""PHONE"""


In [ ]:
total_person_depth = preprocess("person_2","person_depth")
total_person_depth

case_id,person_depth_num_group1,person_depth_num_group2,person_depth_addres_district_368M,person_depth_addres_role_871L,person_depth_addres_zip_823M,person_depth_conts_role_79M,person_depth_empls_economicalst_849M,person_depth_empls_employedfrom_796D,person_depth_empls_employer_name_740M,person_depth_relatedpersons_role_762T
u32,u16,u16,str,str,str,str,str,date,str,str
"843,132",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"887,705",2,1,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"1,661,597",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
…,…,…,…,…,…,…,…,…,…,…
"57,631",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"57,569",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"57,636",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null


In [ ]:
total_other_shallow = preprocess("other_1","other_shallow")
total_other_shallow

case_id,other_shallow_amtdebitincoming_4809443A,other_shallow_amtdebitoutgoing_4809440A,other_shallow_amtdepositbalance_4809441A,other_shallow_amtdepositincoming_4809444A,other_shallow_amtdepositoutgoing_4809442A,other_shallow_num_group1
u32,f32,f32,f32,f32,f32,u16
"215,261",0,0,0,0,2.799999952316284,0
"237,534","25,000","25,000",0,0,0,0
"1,840,115","4,000","4,000",0,0,0,0
…,…,…,…,…,…,…
"57,823",0,0,"101,817",0,"33,950.80078125",0
"57,774",0,0,988.6000366210938,0,8.199999809265137,0
"57,635",0,0,"12,028.7998046875",0,"4,011",0


In [ ]:
total_deposit_shallow = preprocess("deposit_1","deposit_shallow")
total_deposit_shallow

case_id,deposit_shallow_amount_416A,deposit_shallow_num_group1,deposit_shallow_contractenddate_991D,deposit_shallow_openingdate_313D
u32,f32,u16,date,date
"173,582",393.0220031738281,0,2017-11-13,2013-11-13
"147,733",827.2440795898438,2,2016-12-17,2016-07-13
"196,245",269.7220153808594,0,2017-12-16,2014-12-17
…,…,…,…,…
"57,951",289.17401123046875,0,2017-10-09,2013-10-10
"57,925",208.1840057373047,0,null,2017-04-07
"57,932","1,008.9960327148438",0,null,2017-07-06


In [ ]:
total_debitcard_shallow = preprocess("debitcard","card_shallow")
total_debitcard_shallow

case_id,card_shallow_last180dayaveragebalance_704A,card_shallow_last180dayturnover_1134A,card_shallow_last30dayturnover_651A,card_shallow_num_group1,card_shallow_openingdate_857D
u32,f32,f32,f32,u16,date
"2,697,761",null,null,null,2,2013-12-26
"2,644,831",null,null,null,0,2014-05-26
"197,701",0,"20,000",0,4,2014-02-21
…,…,…,…,…,…
"58,038",null,null,null,1,2016-03-04
"57,925",null,null,null,0,2017-04-07
"57,719",null,null,null,0,2016-09-22


In [ ]:
total_credit_internal_shallow = preprocess("bureau_a_1","int_shallow")
total_credit_internal_shallow

case_id,int_shallow_annualeffectiverate_199L,int_shallow_annualeffectiverate_63L,int_shallow_contractsum_5085717L,int_shallow_credlmt_230A,int_shallow_credlmt_935A,int_shallow_debtoutstand_525A,int_shallow_debtoverdue_47A,int_shallow_dpdmax_139P,int_shallow_dpdmax_757P,int_shallow_instlamount_768A,int_shallow_instlamount_852A,int_shallow_interestrate_508L,int_shallow_monthlyinstlamount_332A,int_shallow_monthlyinstlamount_674A,int_shallow_nominalrate_281L,int_shallow_nominalrate_498L,int_shallow_num_group1,int_shallow_numberofcontrsvalue_258L,int_shallow_numberofcontrsvalue_358L,int_shallow_numberofinstls_229L,int_shallow_numberofinstls_320L,int_shallow_numberofoutstandinstls_520L,int_shallow_numberofoutstandinstls_59L,int_shallow_numberofoverdueinstlmax_1039L,int_shallow_numberofoverdueinstlmax_1151L,int_shallow_numberofoverdueinstls_725L,int_shallow_numberofoverdueinstls_834L,int_shallow_outstandingamount_354A,int_shallow_outstandingamount_362A,int_shallow_overdueamount_31A,int_shallow_overdueamount_659A,int_shallow_overdueamountmax2_14A,int_shallow_overdueamountmax2_398A,int_shallow_overdueamountmax_155A,int_shallow_overdueamountmax_35A,int_shallow_periodicityofpmts_1102L,…,int_shallow_totalamount_6A,int_shallow_totalamount_996A,int_shallow_totaldebtoverduevalue_178A,int_shallow_totaldebtoverduevalue_718A,int_shallow_totaloutstanddebtvalue_39A,int_shallow_totaloutstanddebtvalue_668A,int_shallow_classificationofcontr_13M,int_shallow_classificationofcontr_400M,int_shallow_contractst_545M,int_shallow_contractst_964M,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_financialinstitution_591M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_426M,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,int_shallow_subjectrole_182M,int_shallow_subjectrole_93M
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,str,str,str,str,date,date,date,date,date,str,str,str,str,str,str,str,date,date,date,date,date,date,str,str,str,str,str,str,date,str,str
"1,301,783",null,null,null,null,"44,798","39,023.75390625",0,10,null,"2,938.688232421875",null,null,"2,938.688232421875",null,43,null,10,2,null,null,18,null,15,17,null,0,null,null,"16,584.6953125",null,0,"2,380.35400390625",null,"2,380.35400390625",null,null,…,null,"18,540",0,null,"39,023.75390625",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2020-05-15,null,null,2014-01-25,null,"""a55475b1""",null,"""12.0""","""2017.0""",null,"""a55475b1""","""a55475b1""",2019-03-05,null,null,2014-10-12,null,2018-12-29,null,"""1.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-03-21,"""a55475b1""","""a55475b1"""
"618,576",null,null,null,null,"20,176","23,115.552734375",0,0,null,"3,362.800048828125",null,null,"3,362.800048828125",null,42.5,null,10,2,null,null,11,null,9,0,null,0,null,null,"2,939.552001953125",null,0,0,null,0,null,null,…,null,"3,374",0,null,"23,115.552734375",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2020-07-20,null,null,2018-10-29,null,"""a55475b1""",null,"""8.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-01-29,null,null,null,null,null,null,"""8.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-02-08,"""a55475b1""","""a5

In [ ]:
total_credit_internal_depth = preprocess("bureau_a_2","int_depth")
total_credit_internal_depth

case_id,int_depth_collater_valueofguarantee_1124L,int_depth_collater_valueofguarantee_876L,int_depth_num_group1,int_depth_num_group2,int_depth_pmts_dpd_1073P,int_depth_pmts_dpd_303P,int_depth_pmts_overdue_1140A,int_depth_pmts_overdue_1152A,int_depth_collater_typofvalofguarant_298M,int_depth_collater_typofvalofguarant_407M,int_depth_collaterals_typeofguarante_359M,int_depth_collaterals_typeofguarante_669M,int_depth_pmts_month_158T,int_depth_pmts_month_706T,int_depth_pmts_year_1139T,int_depth_pmts_year_507T,int_depth_subjectroles_name_541M,int_depth_subjectroles_name_838M
u32,f32,f32,u16,u16,f32,f32,f32,f32,str,str,str,str,str,str,str,str,str,str
"1,251,053",0,null,0,23,0,null,0,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""5.0""",null,"""2018.0""",null,"""a55475b1""","""a55475b1"""
"630,628","16,188",null,1,35,0,null,0,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""7.0""",null,"""2018.0""",null,"""a55475b1""","""a55475b1"""
"642,448",0,null,2,35,21,null,"1,229.3919677734375",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""11.0""",null,"""2018.0""",null,"""a55475b1""","""a55475b1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"82,797",0,0,0,9,0,0,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""4.0""","""2.0""","""2020.0""","""2018.0""","""a55475b1""","""a55475b1"""
"85,784",0,null,0,9,0,null,0,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""2.0""",null,"""2020.0""",null,"""a55475b1""","""a55475b1"""
"91,257",0,0,0,9,0,0,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""10.0""","""10.0""","""2020.0""","""2017.0""","""a55475b1""","""a55475b1"""


In [ ]:
total_credit_external_shallow = preprocess("bureau_b_1","ext_shallow")
total_credit_external_shallow

case_id,ext_shallow_amount_1115A,ext_shallow_credlmt_1052A,ext_shallow_credlmt_228A,ext_shallow_credlmt_3940954A,ext_shallow_credquantity_1099L,ext_shallow_credquantity_984L,ext_shallow_debtpastduevalue_732A,ext_shallow_debtvalue_227A,ext_shallow_dpd_550P,ext_shallow_dpd_733P,ext_shallow_dpdmax_851P,ext_shallow_installmentamount_644A,ext_shallow_installmentamount_833A,ext_shallow_instlamount_892A,ext_shallow_interesteffectiverate_369L,ext_shallow_interestrateyearly_538L,ext_shallow_maxdebtpduevalodued_3940955A,ext_shallow_num_group1,ext_shallow_numberofinstls_810L,ext_shallow_overdueamountmax_950A,ext_shallow_pmtdaysoverdue_1135P,ext_shallow_pmtnumpending_403L,ext_shallow_residualamount_1093A,ext_shallow_residualamount_127A,ext_shallow_residualamount_3940956A,ext_shallow_totalamount_503A,ext_shallow_totalamount_881A,ext_shallow_classificationofcontr_1114M,ext_shallow_contractdate_551D,ext_shallow_contractmaturitydate_151D,ext_shallow_contractst_516M,ext_shallow_contracttype_653M,ext_shallow_credor_3940957M,ext_shallow_dpdmaxdatemonth_804T,ext_shallow_dpdmaxdateyear_742T,ext_shallow_lastupdate_260D,ext_shallow_overdueamountmaxdatemonth_494T,ext_shallow_overdueamountmaxdateyear_432T,ext_shallow_periodicityofpmts_997L,ext_shallow_periodicityofpmts_997M,ext_shallow_pmtmethod_731M,ext_shallow_purposeofcred_722M,ext_shallow_subjectrole_326M,ext_shallow_subjectrole_43M
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,date,date,str,str,str,str,str,date,str,str,str,str,str,str,str,str
"188,522","612,455.875","20,000","40,000","20,000",2,9,0,"559,850.625",0,0,"28,517",0,"563,349.8125","6,357.60205078125",4.590000152587891,39,0.6000000238418579,2,120,2,0,107,0,"19,986.291015625","19,986.291015625","626,453.875","1,523,398.375","""ea6782cc""",2019-04-26,2020-01-02,"""7241344e""","""4257cbed""","""P0_31_66""","""8.0""","""2019.0""",2019-12-20,"""5.0""","""2019.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
"1,826,601",null,0,null,0,1,null,0,null,0,null,0,null,0,null,null,39,0,0,null,0,0,null,null,0,0,0,null,"""ea6782cc""",2019-03-14,2022-01-14,"""7241344e""","""1c9c5356""","""P0_31_66""","""4.0""","""2019.0""",2020-04-04,"""4.0""","""2019.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""a55475b1"""
"1,336,900","101,280","100,000",0,"100,000",2,5,187.68800354003906,"2,880.93212890625",844,0,"16,378",0,"81,657.515625","3,657.60009765625",22.950000762939453,45,1.2000000476837158,2,36,4,0,3,0,"81,657.515625","81,657.515625","117,080","170,753.40625","""ea6782cc""",2018-07-07,2019-04-14,"""7241344e""","""4257cbed""","""P0_31_66""","""4.0""","""2018.0""",2019-04-17,"""1.0""","""2018.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,660",null,"133,279",null,"113,279",2,1,0,null,0,0,"32,618",0,"109,099.578125",null,null,null,11.40000057220459,1,null,16.200000762939453,0,null,null,"109,099.578125","90,845.9765625","133,279","16,800","""ea6782cc""",2019-12-08,2021-03-14,"""7241344e""","""1c9c5356""","""b619fa46""","""10.0""","""2017.0""",2020-10-10,"""12.0""","""2019.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""a55475b1""","""a55475b1"""
"57,689",null,"144,958.796875",null,"114,958.8046875",2,null,0,null,0,null,0,null,"82,776.8046875",null,null,null,0,1,null,0,0,null,null,"82,776.8046875","72,465.8046875","144,958.796875",null,"""ea6782cc""",2019-08-20,2021-12-31,"""7241344e""","""1c9c5356""","""b619fa46""","""2.0""","""2020.0""",2020-10-10,"""2.0""","""2020.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""a55475b1""","""a55475b1"""
"57,700","425,600",0,null,0,2,null,"6,062.72607421875","678,996","27,282",null,"27,911",null,"843,961.6875","18,861",null,null,4,2,53,4,3,36,null,0,0,"625,600",null,"""ea6782cc""",2020-10-12,2023-10-12,"""8f3a197f""","""4257cbed""","""74bd67a8"""

In [ ]:
total_credit_external_depth = preprocess("bureau_b_2","ext_depth")
total_credit_external_depth

case_id,ext_depth_num_group1,ext_depth_num_group2,ext_depth_pmts_dpdvalue_108P,ext_depth_pmts_pmtsoverdue_635A,ext_depth_pmts_date_1107D
u32,u16,u16,f32,f32,date
"2,575,210",1,36,0,0,2018-11-15
"855,690",1,8,0,0,2019-10-15
"104,753",2,35,"52,730",2,2018-10-15
…,…,…,…,…,…
"1,686,588",1,6,0,0,2019-09-15
"1,946,270",1,35,0,0,2020-09-15
"57,660",1,34,"8,814",2.200000047683716,2020-08-15


In [ ]:
total_registry_a = preprocess("registry_a","reg_a")
total_registry_a

case_id,reg_a_amount_4527230A,reg_a_num_group1,reg_a_name_4527232M,reg_a_recorddate_4527225D
u32,f32,u16,str,date
"1,552,673","4,837.60009765625",5,"""88a34b2a""",2019-10-09
"939,084","1,596.2000732421875",8,"""1dfc94c5""",2020-02-01
"857,527","3,412.800048828125",15,"""d4ad0a11""",2019-11-24
…,…,…,…,…
"1,704,672","2,596",5,"""5670e628""",2020-01-06
"57,679","3,640",4,"""ba006408""",2020-10-20
"57,689","3,214.800048828125",6,"""aac4edf4""",2020-10-20


In [ ]:
total_registry_b = preprocess("registry_b","reg_b")
total_registry_b

case_id,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_b_name_4917606M
u32,f32,u16,date,str
"240,940","10,135",5,2020-06-05,"""b0356c56"""
"1,946,693","8,910",5,2020-05-07,"""cb8cda49"""
"985,345","18,630",5,2020-02-03,"""4e7d21f5"""
…,…,…,…,…
"1,899,732","20,873.80078125",5,2020-07-14,"""f9a186d6"""
"57,549","8,760.6005859375",11,2020-08-06,"""bf93e092"""
"57,543","31,060.201171875",4,2020-09-17,"""d2142f5c"""


In [ ]:
total_registry_c = preprocess("registry_c","reg_c")
total_registry_c

case_id,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_employername_160M,reg_c_processingdate_168D
u32,u16,f32,str,date
"135,303",9,"1,331.5999755859375","""18ac4741""",2019-04-09
"718,866",4,"1,400","""b37ec2f9""",2019-06-10
"1,365,205",12,"4,499.60009765625","""d4fe4906""",2019-03-29
…,…,…,…,…
"1,530,489",1,"5,535.80029296875","""09de35c5""",2019-09-24
"701,130",5,850,"""527d1b27""",2019-04-19
"18,197",2,850,"""587ca2ba""",2019-04-25


In [ ]:
drop_columns = [
    "Date",
    "static_base_commnoinclast6m_3546845L",
    "static_base_deferredmnthsnum_166L",
    "static_base_mastercontrelectronic_519L",
    "static_base_mastercontrexist_109L",
    "static_base_bankacctype_710L",
    "static_base_isdebitcard_729L",
    "static_base_paytype1st_925L",
    "static_base_paytype_783L",
    "static_base_typesuite_864L",
    "person_shallow_contaddr_matchlist_1032L",
    "person_shallow_remitter_829L",
    "person_depth_empls_employer_name_740M",
    "int_shallow_subjectrole_182M",
    "person_shallow_relationshiptoclient_642T",
    "int_shallow_refreshdate_3813885D",
    "int_shallow_monthlyinstlamount_674A",
    "static_base_maxlnamtstart6m_4525199A",
    "static_base_dtlastpmtallstes_4499206D",
    "static_base_cardtype_51L",
    "static_base_applicationscnt_1086L",
    "past_shallow_profession_152M",
    "int_shallow_periodicityofpmts_1102L",
    "past_shallow_annuity_853A",
    "card_shallow_openingdate_857D",
    "deposit_shallow_openingdate_313D",
    "int_shallow_credlmt_230A",
    "static_base_avgpmtlast12m_4525200A",
    "int_shallow_monthlyinstlamount_332A",
    "static_base_maxinstallast24m_3658928A",
    "int_shallow_outstandingamount_354A",
    "static_base_clientscnt_946L",
    "int_shallow_credlmt_935A",
    "int_shallow_dpdmaxdatemonth_442T",
    "int_shallow_periodicityofpmts_837L",
    "static_base_clientscnt_304L",
    "int_shallow_totaloutstanddebtvalue_668A",
    "int_shallow_residualamount_488A",
    "person_shallow_contaddr_smempladdr_334L",
    "person_depth_conts_role_79M",
    "int_shallow_subjectrole_93M",
    "int_shallow_contractsum_5085717L",
    "past_depth_cacccardblochreas_147M",
    "int_shallow_overdueamount_659A",
    "int_shallow_debtoverdue_47A",
    "int_shallow_totaldebtoverduevalue_178A",
    "int_shallow_overdueamountmaxdatemonth_284T",
    "person_shallow_empladdr_district_926M",
    "person_depth_empls_economicalst_849M",
    "static_base_maxpmtlast3m_4525190A",
    "past_shallow_isbidproduct_390L",
    "person_shallow_housetype_905L",
    "person_shallow_empladdr_zipcode_114M",
    "int_shallow_overdueamountmax_155A",
    "int_shallow_overdueamountmax2_14A",
    "int_shallow_financialinstitution_591M",
    "static_base_totinstallast1m_4525188A",
    "static_base_lastapprcommoditycat_1041M",
    "static_base_applicationcnt_361L",
    "person_shallow_safeguarantyflag_411L",
    "person_depth_addres_zip_823M",
    "person_depth_addres_district_368M",
    "int_shallow_overdueamountmax_35A",
    "static_base_clientscnt_493L",
    "int_shallow_overdueamountmax2_398A",
    "past_shallow_actualdpd_943P",
    "past_depth_credacc_cards_status_52L",
    "person_depth_num_group2",
    "static_base_actualdpdtolerance_344P",
    ]

In [ ]:
gain_df = pl.read_csv("/home/sohail/Downloads/gains.csv")
drop_list = gain_df.filter(pl.col("gain") < 5000)["col_name"].to_list()
select_list = gain_df.filter(pl.col("gain") > 20000)["col_name"].to_list()

In [ ]:
df = (
    total_base
    .join(
        total_past_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_base,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_other_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_deposit_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_debitcard_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_a,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_b,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_c,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .with_columns(
        (pl.col(pl.Date) - pl.col("Date")).dt.total_days(),
        pl.col(pl.String).cast(pl.Categorical)
    )
    .drop("Date")
    # .with_columns(
    #     pl.col(pl.INTEGER_DTYPES).fill_null(strategy="mean"),
    #     pl.col(pl.FLOAT_DTYPES).fill_null(strategy="mean"),
    #     pl.col(pl.Boolean).fill_null(value=pl.col(pl.Boolean).drop_nulls().mode().first()),
    #     pl.col(pl.Categorical).fill_null(value=pl.col(pl.Categorical).drop_nulls().mode().first())
    # )
)

In [ ]:
df.write_parquet("/home/sohail/Downloads/total_df.parquet")

In [30]:
# corr_df = pd.DataFrame()
# for i,col in enumerate(total_df.columns):
#     corr_df.loc[i,"col"] = col
#     corr_df.loc[i,"val"] = (
#         total_df
#         .select(pl.corr("target",col))
#         .item()
#     )
# corr_df = pl.from_pandas(corr_df).sort(by="val")
# cols_list = corr_df.filter((pl.col("val") > -0.0025) & (pl.col("val") < 0.0025))["col"].to_list()

In [31]:
test_base.select("case_id_base").to_series().to_list()

[57543, 57549, 57551, 57552, 57569, 57630, 57631, 57632, 57633, 57634]

In [32]:
submission_df = df[df["case_id_base"].isin(test_base.select("case_id_base").sort(by="case_id_base").to_series().to_list())].drop(columns=["target"])
submission_df

AttributeError: 'Series' object has no attribute 'isin'

In [ ]:
X_total = df.loc[~df["case_id_base"].isin(test_base.select("case_id_base").sort(by="case_id_base").to_series().to_list())].drop(columns=["target","case_id_base"])
y_total = df.loc[~df["case_id_base"].isin(test_base.select("case_id_base").sort(by="case_id_base").to_series().to_list()),["target"]]

In [ ]:
del df
del total_past_shallow
del total_past_depth
del total_static_base
del total_person_depth
del total_person_shallow
del total_other_shallow
del total_deposit_shallow
del total_debitcard_shallow
del total_credit_external_depth
del total_credit_external_shallow
del total_credit_internal_depth
del total_credit_internal_shallow
del total_registry_a
del total_registry_b
del total_registry_c
gc.collect()

27

In [ ]:
params = {
    "objective":"binary",
    "metric":"auc",
    "max_depth": 16,
    "learning_rate": 0.05,
    "n_estimators": 2000,
    "colsample_bynode": 0.8,
    "colsample_bytree": 0.8,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees": True,
    "num_leaves": 64,
    "device": "gpu",
    "verbose": -1
}

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(X_total,y_total,stratify=y_total,random_state=69,shuffle=True,test_size=0.15)
del X_total,y_total

In [ ]:
model = lgb.LGBMClassifier(**params)
model.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],callbacks=[lgb.log_evaluation(100),lgb.early_stopping(500)])

Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.832495
[200]	valid_0's auc: 0.843628
[300]	valid_0's auc: 0.848076
[400]	valid_0's auc: 0.850703
[500]	valid_0's auc: 0.85213
[600]	valid_0's auc: 0.853176
[700]	valid_0's auc: 0.853795
[800]	valid_0's auc: 0.854136
[900]	valid_0's auc: 0.854428
[1000]	valid_0's auc: 0.854753
[1100]	valid_0's auc: 0.854902
[1200]	valid_0's auc: 0.855
[1300]	valid_0's auc: 0.855121
[1400]	valid_0's auc: 0.855187
[1500]	valid_0's auc: 0.85526
[1600]	valid_0's auc: 0.855392
[1700]	valid_0's auc: 0.855437
[1800]	valid_0's auc: 0.855495
[1900]	valid_0's auc: 0.855499
[2000]	valid_0's auc: 0.855426
Did not meet early stopping. Best iteration is:
[1823]	valid_0's auc: 0.85554


LGBMClassifier(colsample_bynode=0.7, colsample_bytree=0.7, device='gpu',
               extra_trees=True, learning_rate=0.05, max_depth=16, metric='auc',
               n_estimators=2000, num_leaves=64, objective='binary',
               random_state=42, reg_alpha=0.1, reg_lambda=10, verbose=-1)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(X_train,y_train)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(100))
model.add(keras.layers.Dense(100))
model.add(keras.layers.Dense(100))
model.add(keras.layers.Dense(100))
model.add(keras.layers.Dense(1))
model.compile(
    loss="binary_crossentropy",
    metrics=[keras.metrics.AUC()],
    optimizer=keras.optimizers.Adam()
)
model.fit(X_train.to_numpy(),y_train.to_numpy(),validation_data=(X_valid.to_numpy(),y_valid.to_numpy()),batch_size=32)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# splits = 3
# models = [
#     lgb.LGBMClassifier(**params) for _ in range(splits)
# ]
# cv = StratifiedGroupKFold(n_splits=splits)
# for i,(train_ind,valid_ind) in enumerate(cv.split(X_total,y_total,groups=X_total["week"])):
#     X_train,y_train = X_total.iloc[train_ind],y_total.iloc[train_ind]
#     X_valid,y_valid = X_total.iloc[valid_ind],y_total.iloc[valid_ind]
#     models[i].fit(X_train,y_train,eval_set=[(X_valid,y_valid)],callbacks=[lgb.log_evaluation(100),lgb.early_stopping(50)])
#     y_pred = models[i].predict_proba(X_valid)[:,1]
#     print(f"<-------- Roc_Auc score for split {i+1} is {roc_auc_score(y_true=y_valid,y_score=y_pred)} -------->")

In [ ]:
# cols = []
# for col in X_train.columns:
#     if "person_shallow" in col:
#         cols.append(col)
# X_train[cols]

In [ ]:
# class Model:

#     def __init__(self,_models:list) -> None:
#         self.models = _models

#     def predict(self,X):
#         return np.mean([_.predict(X) for _ in self.models],axis=0)
    
#     def predict_proba(self,X):
#         return np.mean([_.predict_proba(X)[:,1] for _ in self.models],axis=0)

In [ ]:
# model = Model(models)
# y_pred = model.predict_proba(X_valid)
# roc_auc_score(y_valid,y_pred)

In [ ]:
sub_df = pd.DataFrame({
    "case_id": submission_df['case_id_base'].to_list(),
    "score": model.predict_proba(submission_df.drop(columns=["case_id_base"]))
}).set_index("case_id")
sub_df.to_csv("./submission.csv")

ValueError: Per-column arrays must each be 1-dimensional